In [34]:
%%backtest
# from datetime import datetime

start = '2020-01-01'
end = '2022-03-21'
universe = StockUniverse('ZZ1000')
benchmark = 'HS300'
freq = 'd'
refresh_rate = Monthly(1)

accounts = {
    'stock_account': AccountConfig(account_type='security', capital_base=1e6)
}


def initialize(context):
    pass


def handle_data(context):
    current_universe = context.get_universe(asset_type='stock',exclude_halt=True)
    today = context.current_date
#     print(today)
    if today.strftime('%d') == '01':
        prev_month_begin = today + pd.tseries.offsets.MonthBegin(-1)
    else:
        prev_month_begin = today + pd.tseries.offsets.MonthBegin(-2)
#     print('prev_month_begin:', prev_month_begin)
    factor_exposure = get_data_cube(symbol=set_universe('ZZ1000'),
                                    field=['d6dk9218qq.size', 'd6dk9218qq.rev',
                                           'd6dk9218qq.illiq', 'd6dk9218qq.ivol'],
                                    start=prev_month_begin, end=today,
                                    style='ast').to_frame().reset_index()
#     print(factor_exposure.columns)
    if factor_exposure.shape[0] == 0:
        pass
    else:
        factor_exposure.columns = ['date', 'sec_id', 'ivol', 'rev', 'size', 'illiq']
#         print(factor_exposure['date'].unique()[-1])
#         factor_exposure = factor_exposure[factor_exposure['date'] == factor_exposure['date'].unique()[-1]]
        ##### Intersection #####
#         stocks_set = {}
#         stocks_set['rev'] = set(factor_exposure.sort_values('rev', ascending=True)['sec_id'].iloc[0:50].tolist())
#         stocks_set['size'] = set(factor_exposure.sort_values('size', ascending=True)['sec_id'].iloc[0:50].tolist())
#         stocks_set['illiq'] = set(factor_exposure.sort_values('illiq', ascending=False)['sec_id'].iloc[0:50].tolist())
#         stocks_set['ivol'] = set(factor_exposure.sort_values('ivol', ascending=True)['sec_id'].iloc[0:50].tolist())
#         buy_list = set.intersection(
#                                     stocks_set['rev'],
# #                                     stocks_set['size'],
#                                     stocks_set['illiq'],
# #                                     stocks_set['ivol']
#         )
#         buy_list = list(buy_list)
        ##### Scoring #####
        factor_exposure['rev_rank'] = factor_exposure['rev'].rank()
        factor_exposure['size_rank'] = factor_exposure['size'].rank()
        factor_exposure['illiq_rank'] = factor_exposure['illiq'].rank(
            ascending=False)
        factor_exposure['ivol_rank'] = factor_exposure['ivol'].rank()
        factor_exposure['rank_sum'] =   factor_exposure['ivol_rank'] + \
                                        factor_exposure['illiq_rank'] + \
                                      factor_exposure['rev_rank'] + \
                                      factor_exposure['size_rank']
        factor_exposure.sort_values('rank_sum', inplace=True)
        nstocks = factor_exposure.shape[0]
        buy_list = list(factor_exposure['sec_id'].iloc[0:20])

        stock_account = context.get_account('stock_account')
        current_positions = stock_account.get_positions(exclude_halt=True)

        for stock in current_positions:
            if stock not in buy_list:
                stock_account.order_to(stock, 0)
        if len(buy_list) == 0:
            pass
        else:
            for stock in buy_list:
                stock_account.order_pct_to(stock, 1/len(buy_list))


Chart()

传入的股票列表中：['688050.XSHG', '688086.XSHG', '300821.XSHE', '605009.XSHG', '300894.XSHE', '300841.XSHE', '688127.XSHG', '002980.XSHE', '301035.XSHE', '688520.XSHG', '688055.XSHG', '688200.XSHG', '688222.XSHG', '688586.XSHG', '300981.XSHE', '002991.XSHE', '603511.XSHG', '300910.XSHE', '300846.XSHE', '688100.XSHG', '688095.XSHG', '300900.XSHE', '688390.XSHG', '003032.XSHE', '300857.XSHE', '601686.XSHG', '002978.XSHE', '605376.XSHG', '688516.XSHG', '605136.XSHG', '002993.XSHE', '301031.XSHE', '605222.XSHG', '605365.XSHG', '605339.XSHG', '605168.XSHG', '603565.XSHG', '601921.XSHG', '300831.XSHE', '605008.XSHG', '688788.XSHG', '688365.XSHG', '688408.XSHG', '605007.XSHG', '605199.XSHG', '605118.XSHG', '600032.XSHG', '003039.XSHE', '688106.XSHG', '301015.XSHE', '688177.XSHG', '002982.XSHE', '603171.XSHG', '688017.XSHG', '603529.XSHG', '603212.XSHG', '688266.XSHG', '002987.XSHE', '002977.XSHE', '003006.XSHE', '688568.XSHG', '300872.XSHE', '600916.XSHG', '601609.XSHG', '688298.XSHG', '688027.XSHG', 

In [35]:
# #查看调仓记录
show_order(start,end)

Order()

In [ ]:
# #查看持仓记录
show_position(start,end)

Position()